##                      Imagen Médica              -                Máster Visión Artificial (2023-2024)

**María Cornejo Antonaya (m.cornejo.2023@alumnos.urjc.es)**

**Nuria Miralles Gavara (n.miralles.2023@alumnos.urjc.es)**

**Juan Montes Cano (juan.montes@urjc.es)**

## Práctica 2: Segmentación de imagen

En la presente práctica, se analizan e implementan diversos algoritmos de segmentación de imagen.


### Ejercicio A) Crecimiento de regiones

El método de crecimiento de regiones se basa en la selección de una semilla en la imagen y la definición de un rango de nivel de gris que caracteriza a la región de interés. A partir de esta semilla y el rango especificado, se procede a expandir iterativamente la región, agregando píxeles adyacentes que cumplen con el criterio de pertenecer al rango de nivel de gris establecido. Este proceso se detiene cuando ya no se pueden agregar más píxeles que cumplan con las condiciones definidas.

En el script el usuario podrá elegir donde se inicializa la semilla, y podrá ver como es incremental la segmentación descomentando las líneas 46 y 47 del script `ejer1.py`.

In [ ]:
import cv2
import numpy as np

def click_event(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        print(f'Coordenada: ({x}, {y})')


def region_grow(image, seed_coords, threshold_min, threshold_max):
    # Create an empty mask to store the region
    mask = np.zeros_like(image, dtype=np.uint8)
    
    # Create a set to store the coordinates of pixels that have been processed
    processed_pixels = set()
    
    # Get the seed pixel value
    seed_value = image[seed_coords[1], seed_coords[0]]
    
    # Create a queue to store the coordinates of pixels to be processed
    queue = []
    queue.append(seed_coords)
    
    # Process the queue until it's empty
    while queue:
        # Get the next pixel coordinates from the queue
        x, y = queue.pop(0)
        
        # Check if the pixel is within the image boundaries and has not been processed before
        if x >= 0 and x < image.shape[1] and y >= 0 and y < image.shape[0] and (x, y) not in processed_pixels:
            # Mark the pixel as processed
            processed_pixels.add((x, y))
            # Check if the pixel value is within the threshold
            if threshold_min <= image[y, x][0] <= threshold_max:
                
                # Set the pixel value in the mask
                mask[y, x] = 255
                
                # Add the neighboring pixels to the queue
                queue.append((x - 1, y))
                queue.append((x + 1, y))
                queue.append((x, y - 1))
                queue.append((x, y + 1))
        #cv2.imshow("Mask", mask)
        #cv2.waitKey(1)

    return mask

# Path to the image file
image_path = "MaterialP2/hueso.tif"

# Read the image using OpenCV
image = cv2.imread(image_path)

# Check if the image was successfully loaded
if image is not None:
    # Display the image
    cv2.imshow("Image", image)
    # Establecer la función de devolución de llamada para el evento de clic del mouse
    cv2.setMouseCallback('Image', click_event)
    # Wait for a key press
    cv2.waitKey(0)
    
    '''
    seed_x = int(input("Enter the x coordinate of the seed pixel: "))
    seed_y = int(input("Enter the y coordinate of the seed pixel: "))'''

    seed_x = 153
    seed_y = 281
    print(seed_x, seed_y)

    # Get the threshold from the user
    '''
    threshold_max = int(input("Enter the threshold value max: "))
    threshold_min = int(input("Enter the threshold value min: "))
    '''



    threshold_max = 180 # 220 te detecta mas hueso
    threshold_min = 110
    
    # Perform region growing
    mask = region_grow(image, (seed_x, seed_y), threshold_min, threshold_max)
    
    # Display the mask
    cv2.imshow("Mask", mask)
    
    # Wait for a key press
    cv2.waitKey(0)
    
    # Close all windows
    cv2.destroyAllWindows()
else:
    print("Failed to load the image.")



### Resultados

Se muestra como la zona ha ido creciendo para realizar la segmentación:


<img src="A1.jpg" alt="Descripción de la imagen" width="700"/>


<img src="A2.jpg" alt="Descripción de la imagen" width="700"/>


<img src="A3.jpg" alt="Descripción de la imagen" width="700"/>


### Ejercicio B) Segmentación usando algoritmo EM

En este apartado se utiliza el algoritmo EM para la segmentación. Considerando el histograma de una imagen formado por gaussianas de cada clase de la imagen, se lleva a cabo la segmentación buscando las gaussianas que definan mejor el histograma. Este algoritmo maximiza la verosimilitud y permite incluir información a priori.

Para desarollar este apartado se ha utilizado la imagen del corte axial de un cerebro facilitado, brain.bmp. Inicialmente, se ha segmentado la imagen con 4 y 6 clases. Obteniendo un resultado muy parecidos, pero en el caso de las 6 clases se realiza la segmentación en menor tiempo dado que con el aumento de clases se divide el histograma en segmentos más pequeños.

<img src="P2_B1.png" alt="Descripción de la imagen" width="300"/>

Para incializar la segmentación se ha modificado el archivo `EMSeg.m`, se han considerado los valores de los vectores de la media, la varianza y proporción. Obteniedno resultados muy parecidos como se puede observar en la siguiente imágen. También, se han consdierado valores aleatorios pero no se ha logrado un resultado con éxito.

<img src="P2_B2.png" alt="Descripción de la imagen" width="300"/>

Por último, no se ha conseguido optimizar el código proporcionado 


## EMSeg.m
function [mask,mu,v,p]=EMSeg(ima,k,mu,v,p)
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%
%   Expectation Maximization image segmentation
%
%   Input:
%          ima: grey color image
%          k: Number of classes
%   Output:
%          mask: clasification image mask
%          mu: vector of class means 
%          v: vector of class variances
%          p: vector of class proportions   
%
%   Example: [mask,mu,v,p]=EMSeg(image,3);
%
%   Author: Prof. Jose Vicente Manjon Herrera
%    Email: jmanjon@fis.upv.es
%     Date: 02-05-2006
%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

% check image
tic;
ima=double(ima);
copy=ima;           % make a copy
ima=ima(:);         % vectorize ima
mi=min(ima);        % deal with negative 
ima=ima-mi+1;       % and zero values
m=max(ima);
s=length(ima);

% create image histogram

h=histogram(ima);
x=find(h);
h=h(x);
x=x(:);h=h(:);

%% initiate parameters
% Si los vectores de media, varianza y porporción no se especifican
% como entrada, se inicialiazan

if (nargin<3)
    mu=(1:k)*m/(k+1);
end

if (nargin < 4)
    v=ones(1,k)*m;
end

if (nargin < 5)
    p=ones(1,k)*1/k;
end 

% start process

sml = mean(diff(x))/1000;
while(1)
        % Expectation
        prb = distribution(mu,v,p,x);
        scal = sum(prb,2)+eps;
        loglik=sum(h.*log(scal));
        
        %Maximizarion
        for j=1:k
                pp=h.*prb(:,j)./scal;
                p(j) = sum(pp);
                mu(j) = sum(x.*pp)/p(j);
                vr = (x-mu(j));
                v(j)=sum(vr.*vr.*pp)/p(j)+sml;
        end
        p = p + 1e-3;
        p = p/sum(p);

        % Exit condition
        prb = distribution(mu,v,p,x);
        scal = sum(prb,2)+eps;
        nloglik=sum(h.*log(scal));                
        if((nloglik-loglik)<0.0001) break; end;        

        clf
        plot(x,h);
        hold on
        plot(x,prb,'g--')
        plot(x,sum(prb,2),'r')
        drawnow
end

% calculate mask
mu=mu+mi-1;   % recover real range
s=size(copy);
mask=zeros(s);


% Obtener la probabilidad de pertenencia a cada clase basada en el histograma
%probabilidad_clases = zeros(s(1), s(2), k);
%for n = 1:k
    %probabilidad_clases(:, :, n) = distribution(mu(n), v(n), p(n), copiar);
%end
for i=1:s(1),
for j=1:s(2),
  for n=1:k
    c(n)=distribution(mu(n),v(n),p(n),copy(i,j)); 
  end
  a=find(c==max(c));  
  mask(i,j)=a(1);
end
end



function y=distribution(m,v,g,x)
x=x(:);
m=m(:);
v=v(:);
g=g(:);
for i=1:size(m,1)
   d = x-m(i);
   amp = g(i)/sqrt(2*pi*v(i));
   y(:,i) = amp*exp(-0.5 * (d.*d)/v(i));
end


function[h]=histogram(datos)
datos=datos(:);
ind=find(isnan(datos)==1);
datos(ind)=0;
ind=find(isinf(datos)==1);
datos(ind)=0;

tam=length(datos);
m=ceil(max(datos))+1;


h=zeros(1,m);
for i=1:tam,
    % tam = 201627
    f=floor(datos(i));    
    if(f>0 & f<(m-1))        
        a2=datos(i)-f;
        a1=1-a2;
        h(f)  =h(f)  + a1;      
        h(f+1)=h(f+1)+ a2;                          
    end;
end;
h=conv(h,[1,2,3,2,1]);
h=h(3:(length(h)-2));
h=h/sum(h);

toc;


In [ ]:
%% Práctica 2. Apartado B: Segmentación usando EM
% Segmentación de imágenes a partir de una estimación
% de máxima verosimilitud de los parámetros del modelo
% estadístico.
% Parámetros de un modelo de mezcla de distribuciones gaussianas

close all;
clear all;
clc;
image = imread("brain.bmp");
gray = rgb2gray(image);

% Segmentación con 4 clases
[mask_4,mu_4,v_4,p_4]=EMSeg(gray,4);

% Segmentación con 6 clases (Volumen)
classes = 6;
[mask_6, mu_6, v_6, p_6]= EMSeg(gray, classes);
time_6 = toc;

% Display the original image and the segmentation mask
figure(1);
subplot(2, 1, 1);
imshow(image);
title('Original Image');
subplot(2, 2, 3);
imshow(double(image).*mask_4);
colormap('jet');
title('Segmentation Mask 4');
subplot(2,2,4);
imshow(double(image).*mask_6);
colormap('jet');
title('Segmentation Mask 6');


%%%%%%
%% B) Estudio de la segmentación con inicialización
classes = 6;
[mask0, mu, v, p] = EMSeg(gray, classes);
[mask1, ~, ~, ~] = EMSeg(gray, classes, mu, v, p);

figure(2);
subplot(2, 2, 1);
imshow(image);
title('Imagen original')
subplot(2, 2, 2);
imshow(double(image).*mask0);
title('Segmentación sin inicializar')
subplot(2,2,3);
imshow(double(image).*mask1);
title('Segmentación con inicialización')
subplot(2,2,4);
imshowpair(mask0,mask1,'diff');
title('Diferencia entre segmentaciones')
sgtitle('Apartado P2/2.B') 


### Ejercicio C) Segmentación usando el algoritmo Watershed

El algoritmo Watershed es una técnica de segmentación de imágenes que se utiliza para separar regiones contiguas en una imagen que están separadas por líneas de contorno. La idea principal del algoritmo es imaginar la intensidad de los píxeles de la imagen como una superficie topográfica, donde los mínimos locales corresponden a los valles y las cuencas de agua representan las regiones que deseamos segmentar. Inicialmente, se marcan los mínimos locales como puntos de inicio de inundación, luego se simula un proceso de inundación desde estos puntos, donde se llenan las cuencas de agua y se encuentran los límites naturales entre las regiones.

De esta manera, el algoritmo Watershed divide la imagen en regiones significativas, creando una segmentación que refleja los cambios abruptos en la intensidad de los píxeles. Sin embargo, este método puede producir una sobresegmentación, donde se generan regiones demasiado pequeñas y numerosas, lo que requiere pasos adicionales, como la imposición de mínimos locales, para obtener una segmentación más precisa y útil.

In [ ]:
% Leer la imagen 
imagen = imread('higado.bmp');

% Imagen en escala de grises
imagen_gris = rgb2gray(imagen);

% Paso 1: Calcular la imagen de gradiente
mascara = fspecial('sobel'); % Crear la mascara de Sobel
gradiente_x = imfilter(imagen_gris, mascara'); % Calcular el gradiente en dirección x
gradiente_y = imfilter(imagen_gris, mascara); % Calcular el gradiente en dirección y
gradiente = sqrt(double(gradiente_x).^2 + double(gradiente_y).^2); % Calcular el modulo de los gradientes

% Paso 2: Aplicar Watershed a la imagen de gradiente
transformada_distancia = bwdist(~imbinarize(gradiente)); % Transformada de distancia
transformada_distancia = -transformada_distancia;
transformada_distancia(~imbinarize(gradiente)) = -Inf; % Establecer los minimos locales
segmentacion = watershed(transformada_distancia); % Aplicar Watershed
segmentacion(~mascara) = 0;

% Paso 3: Utilizar imimposemin para imponer mínimos en las zonas adecuadas
% Operaciones de morfologia matematica para limpiar la imagen y mejorar la segmentacion
I = imagen_gris;
se = strel("disk",20);
Io = imopen(I,se);
Ie = imerode(I,se);
Iobr = imreconstruct(Ie,I);
Ioc = imclose(Io,se);
Iobrd = imdilate(Iobr,se);
Iobrcbr = imreconstruct(imcomplement(Iobrd),imcomplement(Iobr));
Iobrcbr = imcomplement(Iobrcbr);

% Identificar maximos regionales
fgm = imregionalmax(Iobrcbr);
se2 = strel(ones(5,5));
fgm2 = imclose(fgm,se2);
fgm3 = imerode(fgm2,se2);
fgm4 = bwareaopen(fgm3,20);

% Binarizar la imagen de fondo
bw = imbinarize(Iobrcbr);
D = bwdist(bw);
DL = watershed(D);
bgm = DL == 0;

% Imponer minimos locales en las regiones adecuadas
minimos = imimposemin(gradiente, bgm | fgm4);

% Paso 4: Obtener el resultado de Watershed sobre la imagen modificada
segmentacion_final = watershed(minimos);

% Visualizacion de los resultados
figure;
subplot(2,2,1); imshow(imagen); title('Imagen Original');
subplot(2,2,2); imshow(gradiente); title('Imagen de Gradiente');
subplot(2,2,3); imshow(label2rgb(segmentacion,'jet',[.5 .5 .5])); title('Resultado Watershed');
subplot(2,2,4); imshow(label2rgb(segmentacion_final)); title('Resultado con mínimos impuestos');


### Resultado

Como se puede observar, al aplicar Watershed directamente sobre la magnitud del gradiente se produce una sobresegamentación. En cambio, al imponer mínimos locales se obtiene un resultado mejorado, diferenciando las diferentes zonas de la imagen. Aun así, el resultado sigue sin ser muy preciso, lo que sugiere que pueden ser necesarios más ajustes o técnicas adicionales para obtener una segmentación más adecuada.

<img src="result_ejer_C.png" alt="Descripción de la imagen" width="700"/>


### Ejercicio D) Segmentación usando el modelo de Chan_Vese

El modelo de segmentación Chan-Vese es un modelo deformable que divide la imagen en dos regiones con máxima diferencia en media de gris. Este modelo evoluciona una superficie en función de las energías medidas, en concreto la diferencia de puntos de dentro y fuera de la región segmentada, junto con una energía medida de la curvatura de la superficie.

Para comenzar, se han analizado y modificado los parámetros del algoritmo proporcionados en los archivos `chanvese.m` y `chanvese_demo.m`; aumentando el valor de la máscara inicial para lograr una mejora en la segmentación de la imagen.

Después, se ha incluido un criterio de parada en la función de los puntos analizados en el presente y el pasado, en comparación con un umbral establecido, con el propósito de determinar si la segmentación se ha estabilizado y si los pasos siguientes pueden generar sobresegmentación.

Por último, se ha limitado el tamaño máximo de la región segmentada para controlar dicha región segmentada y evitar que abarque zonas no deseadas.

<img src="P2_D.png" alt="Descripción de la imagen" width="300"/>

In [ ]:
## chanvese.m
% Region Based Active Contour Segmentation
%
% seg = chanvese(I,init_mask,max_its,alpha,display)
%
% Inputs: I           2D image
%         init_mask   Initialization (1 = foreground, 0 = bg)
%         max_its     Number of iterations to run segmentation for
%         alpha       (optional) Weight of smoothing term
%                       higer = smoother.  default = 0.2
%         display     (optional) displays intermediate outputs
%                       default = true
%
% Outputs: seg        Final segmentation mask (1=fg, 0=bg)
%
% Description: This code implements the paper: "Active Contours Without
% Edges" By Chan Vese. This is a nice way to segment images whose
% foregrounds and backgrounds are statistically different and homogeneous.
%
% Example:
% img = imread('tire.tif');
% m = zeros(size(img));
% m(33:33+117,44:44+128) = 1;
% seg = region_seg(img,m,500);
%
% Coded by: Shawn Lankton (www.shawnlankton.com)
%------------------------------------------------------------------------
function seg = chanvese(I,init_mask,max_its,alpha, thershold, max_area,display)

% Nuevas entradas:
%    thershold      Umbral para controlar la segmentación
%    max_area       Máxima área del objeto segmentado

%% NARGIN
  %-- default value for parameter alpha is .1
  if(~exist('alpha','var')) 
    alpha = .2; 
  end
  %-- default behavior is to display intermediate outputs
  if(~exist('display','var'))
    display = true;
  end
   if(~exist('threshold','var'))
    threshold = 0.01;
   end

  %-- ensures image is 2D double matrix
  I = im2graydouble(I);    
  
  %-- Create a signed distance map (SDF) from mask
  phi = mask2phi(init_mask);
  
  %% Inicialización de las medias previas de interior y exterior
  u0 = 0;
  v0 = 0;

  %--main loop
  for its = 1:max_its   % Note: no automatic convergence test

    idx = find(phi <= 1.2 & phi >= -1.2);  %get the curve's narrow band
    
    %-- find interior and exterior mean
    upts = find(phi<=0);                 % interior points
    vpts = find(phi>0);                  % exterior points
    u = sum(I(upts))/(length(upts)+eps); % interior mean
    v = sum(I(vpts))/(length(vpts)+eps); % exterior mean

    %% Limitar el tamaño máximo de la región segmentada
    area = length(upts);
    if area > max_area
        break;
    end

    %% Criterio de parada en función de puntos medidos
    
    if abs(u - u0) < thershold && abs(v - v0) < thershold
        break;
    end

    u0 = u;
    v0 = v;
    
    F = (I(idx)-u).^2-(I(idx)-v).^2;         % force from image information
    curvature = get_curvature(phi,idx);  % force from curvature penalty
    
    dphidt = F./max(abs(F)) + alpha*curvature;  % gradient descent to minimize energy
    
    %-- maintain the CFL condition
    dt = .45/(max(dphidt)+eps);
        
    %-- evolve the curve
    phi(idx) = phi(idx) + dt.*dphidt;

    %-- Keep SDF smooth
    phi = sussman(phi, .5);

    %-- intermediate output
    if((display>0)&&(mod(its,20) == 0)) 
      showCurveAndPhi(I,phi,its);  
    end
  end
  
  %-- final output
  if(display)
    showCurveAndPhi(I,phi,its);
  end
  
  %-- make mask from SDF
  seg = phi<=0; %-- Get mask from levelset

  
%---------------------------------------------------------------------
%---------------------------------------------------------------------
%-- AUXILIARY FUNCTIONS ----------------------------------------------
%---------------------------------------------------------------------
%---------------------------------------------------------------------
  
  
%-- Displays the image with curve superimposed
function showCurveAndPhi(I, phi, i)
  imshow(I,'initialmagnification',200,'displayrange',[0 255]); hold on;
  contour(phi, [0 0], 'g','LineWidth',4);
  contour(phi, [0 0], 'k','LineWidth',2);
  hold off; title([num2str(i) ' Iterations']); drawnow;
  
%-- converts a mask to a SDF
function phi = mask2phi(init_a)
  phi=bwdist(init_a)-bwdist(1-init_a)+im2double(init_a)-.5;
  
%-- compute curvature along SDF
function curvature = get_curvature(phi,idx)
    [dimy, dimx] = size(phi);        
    [y x] = ind2sub([dimy,dimx],idx);  % get subscripts

    %-- get subscripts of neighbors
    ym1 = y-1; xm1 = x-1; yp1 = y+1; xp1 = x+1;

    %-- bounds checking  
    ym1(ym1<1) = 1; xm1(xm1<1) = 1;              
    yp1(yp1>dimy)=dimy; xp1(xp1>dimx) = dimx;    

    %-- get indexes for 8 neighbors
    idup = sub2ind(size(phi),yp1,x);    
    iddn = sub2ind(size(phi),ym1,x);
    idlt = sub2ind(size(phi),y,xm1);
    idrt = sub2ind(size(phi),y,xp1);
    idul = sub2ind(size(phi),yp1,xm1);
    idur = sub2ind(size(phi),yp1,xp1);
    iddl = sub2ind(size(phi),ym1,xm1);
    iddr = sub2ind(size(phi),ym1,xp1);
    
    %-- get central derivatives of SDF at x,y
    phi_x  = -phi(idlt)+phi(idrt);
    phi_y  = -phi(iddn)+phi(idup);
    phi_xx = phi(idlt)-2*phi(idx)+phi(idrt);
    phi_yy = phi(iddn)-2*phi(idx)+phi(idup);
    phi_xy = -0.25*phi(iddl)-0.25*phi(idur)...
             +0.25*phi(iddr)+0.25*phi(idul);
    phi_x2 = phi_x.^2;
    phi_y2 = phi_y.^2;
    
    %-- compute curvature (Kappa)
    curvature = ((phi_x2.*phi_yy + phi_y2.*phi_xx - 2*phi_x.*phi_y.*phi_xy)./...
              (phi_x2 + phi_y2 +eps).^(3/2)).*(phi_x2 + phi_y2).^(1/2);        
  
%-- Converts image to one channel (grayscale) double
function img = im2graydouble(img)    
  [dimy, dimx, c] = size(img);
  if(isfloat(img)) % image is a double
    if(c==3) 
      img = rgb2gray(uint8(img)); 
    end
  else           % image is a int
    if(c==3) 
      img = rgb2gray(img); 
    end
    img = double(img);
  end

%-- level set re-initialization by the sussman method
function D = sussman(D, dt)
  % forward/backward differences
  a = D - shiftR(D); % backward
  b = shiftL(D) - D; % forward
  c = D - shiftD(D); % backward
  d = shiftU(D) - D; % forward
  
  a_p = a;  a_n = a; % a+ and a-
  b_p = b;  b_n = b;
  c_p = c;  c_n = c;
  d_p = d;  d_n = d;
  
  a_p(a < 0) = 0;
  a_n(a > 0) = 0;
  b_p(b < 0) = 0;
  b_n(b > 0) = 0;
  c_p(c < 0) = 0;
  c_n(c > 0) = 0;
  d_p(d < 0) = 0;
  d_n(d > 0) = 0;
  
  dD = zeros(size(D));
  D_neg_ind = find(D < 0);
  D_pos_ind = find(D > 0);
  dD(D_pos_ind) = sqrt(max(a_p(D_pos_ind).^2, b_n(D_pos_ind).^2) ...
                       + max(c_p(D_pos_ind).^2, d_n(D_pos_ind).^2)) - 1;
  dD(D_neg_ind) = sqrt(max(a_n(D_neg_ind).^2, b_p(D_neg_ind).^2) ...
                       + max(c_n(D_neg_ind).^2, d_p(D_neg_ind).^2)) - 1;
  
  D = D - dt .* sussman_sign(D) .* dD;
  
%-- whole matrix derivatives
function shift = shiftD(M)
  shift = shiftR(M')';

function shift = shiftL(M)
  shift = [ M(:,2:size(M,2)) M(:,size(M,2)) ];

function shift = shiftR(M)
  shift = [ M(:,1) M(:,1:size(M,2)-1) ];

function shift = shiftU(M)
  shift = shiftL(M')';
  
function S = sussman_sign(D)
  S = D ./ sqrt(D.^2 + 1);    





In [ ]:
## chanvese_demo.m
% Demo of "Chan Vese Level Sets"
%
% Example:
% seg_demo
%
% Coded by: Shawn Lankton (www.shawnlankton.com)

clc;
close all;
clear all;
I = imread('ec.jpg');  %-- load the image
m = zeros(size(I,1),size(I,2));          %-- create initial mask
m(110:150,110:150) = 1; 
%m(100:125,100:125) = 1; % Inicial



subplot(2,2,1); imshow(I); title('Input Image');
subplot(2,2,2); imshow(m); title('Initialization');
subplot(2,2,3); title('Segmentation');

% Inicial = 800, 3.0
 %seg = chanvese(I,init_mask,max_its,alpha, thershold, max_area,display)
seg = chanvese(I, m, 600, 1, 0.001, 11111); %-- Run segmentation
%seg = chanvese(I, m, 800, 3.0); %Inicial

subplot(2,2,4); imshow(seg); title('Global Region-Based Segmentation');




### Ejercicio E) Segmentación usando atlas

En este apartado, abordaremos la segmentación de imágenes médicas utilizando un enfoque basado en atlas. Nos centraremos en cómo segmentar una imagen utilizando otra imagen ya etiquetada como referencia. La tarea consiste en desarrollar un código que segmente la imagen MR mediante la asignación de etiquetas basadas en la similitud entre parches de las imágenes. En el contexto de imágenes del cerebro, se buscarán parches en una vecindad de 11x11x11 alrededor de cada punto de interés, utilizando un kernel de 3x3x3 para evaluar la similitud.

El algoritmo es muy lento, y para evaluar regiones suficientemente grandes no es viable, por lo que se ha escogido una ROI dentro del volumen tridimensional sobre el aplicamos la segmentación.

El algoritmo toma un punto a clasificar, y en una vecindad de 11x11x11 en el mismo punto en la imagen del atlas, busca la mayor similitud en kernels de 3x3x3. Esta similitud se realiza usando la diferencia media en dichos kernels con respecto al punto original. El punto que tenga la menor diferencia será el que de la etiqueta para clasificar.


In [ ]:
import SimpleITK as sitk
import matplotlib.pyplot as plt
import numpy as np
# Leer las imágenes
mr_image = sitk.ReadImage("MaterialP2/MR.nii")
ft1_image = sitk.ReadImage("MaterialP2/fT1.nii")
flabels_image = sitk.ReadImage("MaterialP2/fLabels.nii")

# Convertir las imágenes a arrays numpy
mr_array = sitk.GetArrayFromImage(mr_image)
#print(mr_array.shape)
ft1_array = sitk.GetArrayFromImage(ft1_image)
flabels_array = sitk.GetArrayFromImage(flabels_image)

# Crear un subplot con 1 fila y 3 columnas
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Las imágenes volumétricas
images = [mr_array, ft1_array, flabels_array]
titles = ['MR', 'fT1', 'fLabels']
index = 15
# Mostrar las primeras tres imágenes en un solo plot
for i in range(3):
    axes[i].imshow(images[i][index, :, :], cmap='gray')  # Seleccionar la imagen correspondiente al elemento 15
    axes[i].set_title(titles[i])
    axes[i].axis('off')

plt.show()

# Definir las coordenadas de la ROI en mr_array (ejemplo)
x_start_roi, y_start_roi, z_start_roi = 20, 50, 50
x_end_roi, y_end_roi, z_end_roi = 30, 150, 150

# Crear una matriz vacía para almacenar los resultados en la ROI
resultados_roi = np.empty((x_end_roi - x_start_roi, y_end_roi - y_start_roi, z_end_roi - z_start_roi))

# Iterar sobre cada voxel dentro de la ROI
for x in range(x_start_roi, x_end_roi):
    for y in range(y_start_roi, y_end_roi):
        for z in range(z_start_roi, z_end_roi):
            # Coordenadas del voxel de referencia en la imagen derecha
            x_ref, y_ref, z_ref = x, y, z
            
            # Definir las coordenadas de la región 11x11x11 en la imagen derecha
            x_start = max(x_ref - 5, x_start_roi)
            x_end = min(x_ref + 6, x_end_roi)
            y_start = max(y_ref - 5, y_start_roi)
            y_end = min(y_ref + 6, y_end_roi)
            z_start = max(z_ref - 5, z_start_roi)
            z_end = min(z_ref + 6, z_end_roi)

            # Inicializar el voxel más similar y su diferencia mínima
            voxel_mas_similar = None
            diferencia_minima = float('inf')

            # Iterar sobre la región 11x11x11 en la imagen derecha
            for xi in range(x_start, x_end):
                for yi in range(y_start, y_end):
                    for zi in range(z_start, z_end):
                        # Definir las coordenadas del kernel 3x3x3 alrededor del voxel de referencia
                        x_kernel_start = max(xi - 1, x_start)
                        x_kernel_end = min(xi + 2, x_end)
                        y_kernel_start = max(yi - 1, y_start)
                        y_kernel_end = min(yi + 2, y_end)
                        z_kernel_start = max(zi - 1, z_start)
                        z_kernel_end = min(zi + 2, z_end)

                        # Calcular la diferencia media en el kernel 3x3x3
                        diferencia_media = np.mean(np.abs(mr_array[x:x+1, y:y+1, z:z+1] - mr_array[x_kernel_start:x_kernel_end, y_kernel_start:y_kernel_end, z_kernel_start:z_kernel_end]))
                        
                        # Actualizar el voxel más similar si encontramos una diferencia mínima
                        if diferencia_media < diferencia_minima:
                            diferencia_minima = diferencia_media
                            voxel_mas_similar = flabels_array[xi, yi, zi]  # Voxel más similar encontrado

            # Almacenar el voxel más similar en los resultados
            resultados_roi[x - x_start_roi, y - y_start_roi, z - z_start_roi] = voxel_mas_similar



# Elementos a mostrar
elementos_a_mostrar = [0, 5, 9]

# Mostrar los elementos seleccionados
for idx, elemento in enumerate(elementos_a_mostrar, 1):
    plt.subplot(1, len(elementos_a_mostrar), idx)
    plt.imshow(resultados_roi[elemento, :, :], cmap='gray')  # Seleccionar el plano específico
    plt.colorbar()
    plt.title(f"Elemento {elemento}")
    plt.axis('off')

plt.show()

### Resultado

Imagen en la ROI `[20:30],[50,150],[50,150]`, se muestran las slices 0,5,9 que se corresponden con las slices 20,25,29 de la imagen original

<img src="ejerE.png" alt="Descripción de la imagen" width="700"/>
